In [1]:
%cd ../
%reload_ext autoreload
%autoreload 2

/home/akali/projects/model-trainer


In [2]:
import importlib
import logging
import json
from model_trainer import loader
from model_trainer.config import get_path

log = logging.getLogger(__name__)
importlib.reload(loader)

[22:19:46] Logging configured [utils.logging]


<module 'model_trainer.loader' from '/projects/model-trainer/model_trainer/loader.py'>

In [3]:
w2v_cbow_path = get_path("wiki-he", "cbow.kv")
w2v_cbow_model = loader.load_kv(w2v_cbow_path)

# w2v_sg_path = get_path("wiki-he", "sg.kv")
# w2v_cbow = loader.load_kv(w2v_cbow_path)

ft_path = get_path("wiki-he", "fasttext.kv")
ft_model = loader.load_kv(ft_path)

[22:19:47] loading KeyedVectors object from ./data/wiki-he/cbow.kv [gensim.utils]
[22:19:47] loading vectors from ./data/wiki-he/cbow.kv.vectors.npy with mmap=None [gensim.utils]
[22:19:47] KeyedVectors lifecycle event {'fname': './data/wiki-he/cbow.kv', 'datetime': '2021-12-11T22:19:47.652156', 'gensim': '4.1.2', 'python': '3.8.10 (default, Sep 28 2021, 16:10:42) \n[GCC 9.3.0]', 'platform': 'Linux-5.11.0-40-generic-x86_64-with-glibc2.29', 'event': 'loaded'} [gensim.utils]
[22:19:47] Vectors loaded [model_trainer.word2vec]
[22:19:47] loading KeyedVectors object from ./data/wiki-he/fasttext.kv [gensim.utils]
[22:19:48] loading vectors_vocab from ./data/wiki-he/fasttext.kv.vectors_vocab.npy with mmap=None [gensim.utils]
[22:19:48] loading vectors_ngrams from ./data/wiki-he/fasttext.kv.vectors_ngrams.npy with mmap=None [gensim.utils]
[22:19:48] setting ignored attribute vectors to None [gensim.utils]
[22:19:48] setting ignored attribute buckets_word to None [gensim.utils]
[22:20:16] FastT

In [4]:
w2v_cbow_vocab = set(w2v_cbow_model.key_to_index.keys())
ft_vocab = set(ft_model.key_to_index.keys())
common_vocab = w2v_cbow_vocab.intersection(ft_vocab)

s1, s2 = len(w2v_cbow_vocab), len(ft_vocab)
print(f"Vocabulary diff: {abs(s1 - s2)}")
print(f"Common vocab: {len(common_vocab)}")

Vocabulary diff: 1
Common vocab: 576850


In [11]:
import random
from model_trainer import inflections
from model_trainer.inflections import InflectionsDictGenerator, reduce_inflections_dict

importlib.reload(inflections)

inflections_generator = InflectionsDictGenerator(model=w2v_cbow_model)

random.seed(123)
words = random.sample(common_vocab, k=100)

all_inflections = inflections_generator.generate_model_inflections(words=words)
reduced_inflections = reduce_inflections_dict(all_inflections)

print(reduced_inflections)
serialized = json.dumps(reduced_inflections)
with open("text.json", "w") as file:
    file.write(serialized)

[22:24:13] Generating inflections...2 [model_trainer.inflections.inflections_dict_generator]


100%|██████████| 100/100 [00:05<00:00, 19.89it/s]

[22:24:18] Total 11 inflections generated [model_trainer.inflections.inflections_dict_generator]
[22:24:18] Function 'generate_model_inflections' took 0:00:05.037952. [utils.logging]
[22:24:18] Reducing dict of size 11 [model_trainer.inflections.inflections_dict_reducer]



100%|██████████| 11/11 [00:00<00:00, 24672.38it/s]

[22:24:18] Function 'reduce_inflections_dict' took 0:00:00.018632. [utils.logging]
{'שהתעללו': 'שהתעלל', 'dreaming': 'dreamin', 'הפסקאות': 'פסקאות', 'ונעזר': 'נעזר', 'בפסטיבלים': 'פסטיבלים', 'ופסטיבלים': 'פסטיבלים', 'ובפסטיבלים': 'פסטיבלים', 'שיהיה': 'יהיה', 'ועמידה': 'עמידה', 'שניתנה': 'ניתנה', 'ואמיצים': 'אמיצים'}
